# Домашняя работа № 1
## Общие правила
- **Дедлайн: 27.09.2023 23:59**
- Можно сдать до 28.09.2023 12:00 со штрафом в 50% от оценки, позже - только с официальным подтверждением от УО.
- Оценивание: каждый пункт 1 балл, при невыполнении первого пункта - ноль за всю работу.

**Важно!**
Изменилась форма сдачи: теперь сдаем в github classroom (как на проге). Там возможно что-то догрузить после дедлайна, но это будет расцениваться как его несоблюдение.

> [Ссылка](https://classroom.github.com/a/d3wAyVs3)

## На оценку 6
1. Выбрать любой корпус текстов и рассказать, октуда вы его взяли и почему именно его (ответ "потому что он мне понравился" тоже валиден):
    - Он должен быть на русском и содержать порядка 1-2 тысяч записей (абзацев, текстов, предложений etc).
    - Это может быть часть любого открытого корпуса, результат краулинга или даже генерации с помощью модели.
    - Будет здорово, если он будет тематический (машины, лингвистика, литератруа etc), но не обязательно.
2. Провести предобработку текста:
    - лемматизация
    - чистка от пунктуации и стоп-слов
    - что-то еще, что вы считаете нужным
3. Реализовать обратный индекс через частоты
4. Реализовать обратный индекс через BM-25
5. Реализовать поиск: на входе текст запроса и вариант индекса (частоты или bm-25), на выходе топ подходящих документов
6. Описать в readme файле, как правильно запустить ваш код. 

## На оценку 8
1. Реализовать частотный и BM-25 индекс руками через словари
2. Реализовать их же через матрицы. Можно и нужно использовать scipy или numpy для матричных операций

## На оценку 10
1. Ваш код должен соотвествовать следующим критериям:
    - Разделение на модули (разные файлы для разных больших блоков логики)
    - Разделение на классы/функции
    - Одна очевидная для пользователя точка входа: я могу запустить ваш код одной функцией, в которую подается текст
    - Осмысленные имена переменных, функций, классов, модулей
2. Подумайте, как можно формально сравнить два способа индексации с точки зрения качества поиска. Опишите, какой способо вы придумали, и используйте его для сравнения частоного и BM-25 индекса.

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import string
import numpy as np
import math
from typing import List, Dict, Tuple
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Я выбрала корпус шуток, взятый с [Kaggle](https://www.kaggle.com/datasets/konstantinalbul/russian-jokes/). В нем больше записей, чем нам нужно, поэтому из него я выбрала только те шутки, чей рейтинг больше или равен 5. 
Почему я выбрала именно его? Во-первых, он тематический и на приколе. Во-вторых, я устала от того, что мы с друзьями вечно вспоминаем какие-то анекдоты, но никто не может их рассказать, потому что помнит лишь самую суть. По-моему, было бы здорово создать поисковик для таких случаев!

In [3]:
df = pd.read_csv('jokes.csv')

In [4]:
good_jokes = list(set(df[df.rating >= 5].text.values.tolist()))
len(good_jokes)

1375

In [5]:
def lemmatization(sent: str) -> List[str]:
    '''
    This function lemmatizes one string
    :param sent: one raw string
    :return: list of lemmatized words of the string
    '''
    lemmatized_string = []
    for word in nltk.word_tokenize(sent, language="russian"):
        lemma = morph.parse(word)[0].normal_form.strip(string.punctuation + '«—»')
        lemma = lemma.replace('ё', 'е')
        if lemma and lemma not in russian_stopwords and lemma not in (string.punctuation + '«—»') and lemma != '...':
            lemmatized_string.append(lemma)
    return lemmatized_string

In [6]:
def sentences_lemmatization(sentences: List[str]) -> List[List[str]]:
    '''
    This function lemmatizes sentences
    :param sentences: list of raw strings
    :return: list of senetences as a list of lemmatized words
    '''
    lemmatized_texts = []
    for sentence in sentences:
        lemmatized_text = lemmatization(sentence)
        lemmatized_texts.append(lemmatized_text)
    return lemmatized_texts

In [7]:
class ManualFreq:
    '''
    This class realizes frequency ranking score for a corpus of texts.
    :attr corpus_size_: int: number of documents in the corpus
    :attr frequency_dict_: Dict[int, Dict[int, int]]: reversed frequency dictionary
    :attr vocabulary_: Dict[str, int]: dictionary with all the terms in the corpus with their indexes in the reversed frequency dictionary
    '''
    def fit(self, corpus: List[List[str]]):
        '''
        This function calculates various reversed frequency dictionary to calculate frequency ranking score.
        :param corpus: list of senetences as a list of lemmatized words
        '''
        frequency_dict, vocabulary = {}, {}
        for i, doc in enumerate(corpus):
            for word in doc:
                if word not in vocabulary:
                    vocabulary[word] = len(vocabulary)
                num_word = vocabulary[word]
                
                if num_word not in frequency_dict:
                    frequency_dict[num_word] = {}
                if i not in frequency_dict[num_word]:
                    frequency_dict[num_word][i] = 0
                
                frequency_dict[num_word][i] += 1
        
        self.corpus_size_ = len(corpus)
        self.frequency_dict_ = frequency_dict
        self.vocabulary_ = vocabulary

    def search(self, query: List[str]) -> List[float]:
        '''
        This function makes a list of scores for each document in the corpus to cater the current search query.
        :param query: search query
        :return: frequency ranking scores for each document in the corpus
        '''
        freq = {}
        for term in query:
            if term in self.vocabulary_:
                for doc, times in self.frequency_dict_[self.vocabulary_[term]].items():
                    if doc in freq:
                        freq[doc] += times
                    else:
                        freq[doc] = times
                        
        scores = [freq[i] if i in freq else 0 for i in range(self.corpus_size_)]
        return scores

In [8]:
class BM25:
    '''
    This class realizes BM25 ranking score for a corpus of texts.
    :attr tf_: List[Dict[str, int]]: term frequency per document
    :attr df_: Dict[str, int]: document frequency per term
    :attr idf_: Dict[str, float]: inverse document frequency per term
    :attr doc_len_: List[int]: number of terms per document
    :attr corpus_: List[List[str]]: the input corpus
    :attr corpus_size_: int: number of documents in the corpus.
    :attr avg_doc_len_: float: average number of terms for documents in the corpus.
    '''
    def __init__(self, k: float=1.5, b: float=0.75):
        '''
        This function intializes the magic parameters.
        :param k1: first magic parameter 
        :param b: second magic parameter
        '''
        self.b = b
        self.k = k

    def fit(self, corpus: List[List[str]]):
        '''
        This function fits the various statistics to calculate BM25 ranking score.
        :param corpus: list of senetences as a list of lemmatized words
        '''
        tf, df, idf, doc_len = [], {}, {}, []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))
            frequencies = {}
            for term in document:
                frequencies[term] = frequencies.get(term, 0) + 1
            tf.append(frequencies)
            
            for term in frequencies:
                df[term] = df.get(term, 0) + 1
                
        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))
        
        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size

    def score(self, query: List[str], index: int) -> float:
        '''
        This function computes score for the current document to cater the current search query.
        :param query: search query
        :param index: index of the current document in the corpus
        :return: BM25 ranking score for the current document
        '''
        score = 0.0
        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term in frequencies:
                freq = frequencies[term]
                score += ((self.idf_[term] * freq * (self.k + 1)) / 
                          (freq + self.k * (1 - self.b + self.b * doc_len / self.avg_doc_len_)))
                
        return score

    def search(self, query: List[str]) -> List[float]:
        '''
        This function makes a list of scores for each document in the corpus to cater the current search query.
        :param query: search query
        :return: BM25 ranking scores for each document in the corpus
        '''
        scores = [self.score(query, index) for index in range(self.corpus_size_)]
        return scores

In [9]:
def get_top_n_jokes(corpus: List[List[str]], current_query: List[str], n: int=10):
    '''
    This function prints top-n most high ranked jokes to cater current search query.
    :param corpus: the input corpus
    :param query: the input search query
    :param n: the quantity of jokes (default top-10)
    '''
    ranker = input('Choose one ranker (BM25 / ManualBM25 / Freq / ManualFreq):\n')
    tokenized_corpus = sentences_lemmatization(corpus)
    tokenized_query = lemmatization(current_query)
    if ranker == 'ManualBM25':
        bm25 = BM25()
        bm25.fit(tokenized_corpus)
        query_scores = bm25.search(tokenized_query)
        top_n = [f'{str(i + 1)}.\n{pair[1]}\n' 
                 for i, pair in enumerate(sorted(zip(query_scores, corpus), key=lambda x: x[0], reverse=True)[:n])]
        
    elif ranker == 'BM25':
        bm25 = BM25Okapi(tokenized_corpus)
        top_n = [f'{str(i + 1)}.\n{joke}\n' 
                 for i, joke in enumerate(bm25.get_top_n(tokenized_query, corpus, n=n))]
        
    elif ranker == 'ManualFreq':
        man_freq = ManualFreq()
        man_freq.fit(tokenized_corpus)
        query_scores = man_freq.search(tokenized_query)
        top_n = [f'{str(i + 1)}.\n{pair[1]}\n' 
                 for i, pair in enumerate(sorted(zip(query_scores, corpus), key=lambda x: x[0], reverse=True)[:n])]
        
    elif ranker == 'Freq':
        preprocessed_jokes = [' '.join(sentence) for sentence in sentences_lemmatization(corpus)]
        cv = CountVectorizer()
        freq_spm = cv.fit_transform(preprocessed_jokes)
        sparce_matrix = freq_spm.toarray()
        query_scores = np.zeros((1375,))
        list_of_terms = cv.get_feature_names()
        for word in query:
            if word in cv.get_feature_names():
                idex = list_of_terms.index(word)
                query_scores += sparce_matrix[:, idex]
        top_n = [f'{str(i + 1)}.\n{pair[1]}\n' 
                 for i, pair in enumerate(sorted(zip(query_scores, corpus), key=lambda x: x[0], reverse=True)[:n])]
        
    else:
        raise ValueError("Ranker must be BM25 / ManualBM25 / Freq / ManualFreq!")
        
    print(''.join(top_n))

In [10]:
get_top_n_jokes(good_jokes, 'Вовочка рассказывает отцу про учительницу')

Choose one ranker (BM25 / ManualBM25 / Freq / ManualFreq):
ManualFreq
1.
Учительница на уроке дает задание:
- Дети на завтра задаю вам сочинить стих!
Hа следующий день... Вовочка выходит к доске:
- Ну давай Вовочка!...
Вовочка:
- Стоит статУя
В лучах заката
С огромным хУем
В руках лопата!
Учительница:
- Вовочка а можно как нибудь без Этого?!
Вовочка с ходу:
- Стоит статУя
В лучах заката
С обрубком хУя
В руках лопата!
Учительница:
- Вовочка а можно СОВСЕМ без ЭТОГО?!!!!
Вовочка: - Стоит статУя
В лучах заката
Совсем без хУя
в руках лопата!
Учительница (обреченно так):
- Вовочка ну я же просила - без ЭТОГО....
Вовочка:
- Стоит лопата
В лучах заката!
CтатУя с хУем
ушла куда-то...


2.
Учительница на уроке дает задание:
- Дети на завтра задаю вам сочинить стих!
Hа следующий день... Вовочка выходит к доске:
- Ну давай Вовочка! ...
Вовочка:
- Стоит статУя
В лучах заката
С огромным хУем
В руках лопата!
Учительница:
- Вовочка а можно как нибудь без Этого?!
Вовочка с ходу:
- Стоит статУя
В л